In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pickle
%matplotlib inline

### Calibrate Camera using the Input Calibration data

In [2]:
# Specify the output directory to save image and create one if does not exist
CALIB_IMAGES = "./camera_cal/*.jpg"

In [3]:
# Prepare object points
nx = 6 # Number of inside corners in x
ny = 9 # Number of inside corners in y
nz = 3 # Number of channels

from util import camera as camera
cameraMatrix, distortionCoeffs = camera.calibrate_camera(CALIB_IMAGES, nx, ny, nz)

### Class to Store Lane information every frame

In [4]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self, n):
        # was the line detected in the last iteration?
        self.detected = False 
        # Number of past frames to be remembered
        self.n_frame = n
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.best_x = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit_p = None  
        #polynomial coefficients for the most recent fit
        self.current_fit_p = []  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
    def update_lane_data(self, fit_p, fit_inds):
        '''
        Update the lane information of current frame.
        '''
        self.current_fit_p.append(fit_p)
        if len(self.current_fit_p) > self.n_frame:
            #Keep latest n_frames
            self.current_fit_p = self.current_fit_p[len(self.current_fit_p)-self.n_frame:]
        self.best_fit_p = np.average(self.current_fit_p, axis=0)
    

### Image Processing Pipeline

In [ ]:
from util import image_process as ip
def image_processing_pipeline(img):
    '''
    Pipeline for Image Traanformation steps involved in Lane detection
    
    :returns: warped image
    '''
    new_image = img.copy()
    # Perform undistortion
    undist = ip.undistort(new_image, cameraMatrix, distortionCoeffs)
    # Perform Binary Thresholding
    thresholded = ip.get_binary_thresholded_image(undist)
    # Apply Perspective Transform
    warped, M_inv = ip.apply_warp(thresholded)
    return (warped, M_inv)

### Lane Processing Pipeline

In [6]:
from util import lane_detection as lane_det
def lane_processing_pipeline(img):
    '''
    Pipeline for identifying lanes within a tranformed image using polyfit algorithms and from previous frame data in video.
    '''
    warped, Minv = image_processing_pipeline(img)

    # If both left and right lines were detected last frame, use polyfit_using_prev_fit, otherwise use polyfit_sliding_window
    if not left_lane.detected or not right_lane.detected:
        left_fit_p, right_fit_p, left_lane_inds, right_lane_inds = lane_det.polyfit_sliding_window(warped)
    else:
        left_fit_p, right_fit_p, left_lane_inds, right_lane_inds = lane_det.polyfit_using_previous_fit(warped, left_lane.best_fit, right_lane.best_fit)
    
    # Update current lane status
    left_lane.update_lane_data(left_fit_p, left_lane_inds)
    right_lane.update_lane_data(right_fit_p, right_lane_inds)
    
     # draw the current best fit if it exists
    if left_lane.best_fit_p is not None and right_lane.best_fit_p is not None:
        img_out = lane_det.shade_lane_area(img, warped, left_lane.best_fit_p, right_lane.best_fit_p, Minv)
        curve_rad = lane_det.measure_radius_of_curvature(warped, left_lane.best_fit_p, right_lane.best_fit_p, left_lane_inds, right_lane_inds)
        center_offset = lane_det.measure_offset_from_center(warped, left_lane.best_fit_p, right_lane.best_fit_p)
        img_out = lane_det.add_distance_text_to_image(img_out, curve_rad, center_offset)      
    else:
        img_out = img
        
    return img_out

### Process Video

In [7]:
from moviepy.editor import VideoFileClip
# Create two Line objects for right and left lanes.
# Number of frame chosen as 10 after trial and error
left_lane = Line(10)
right_lane = Line(10)
video_output1 = 'project_video_output.mp4'
video_input1 = VideoFileClip('project_video.mp4')#.subclip(22,26)
processed_video = video_input1.fl_image(lane_processing_pipeline)
%time processed_video.write_videofile(video_output1, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [04:13<00:00,  4.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 12min 39s, sys: 20.3 s, total: 12min 59s
Wall time: 4min 13s
